# Лабораторная работа № 2

## Машинное обучение на больших данных с использованием фреймворка Apache Spark и библиотеки SparkML

### Часть 1

В данной части работы рассмотрены:
- подготовка признаков для рашения задачи **линейной регрессии**;
- создание и обучение модели линейной регрессии;
- оценка качества модели.ели.

### Запуск `Spark`-сессии

Подключаем необходимые библиотеки.

In [1]:
import os
import csv
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, MinMaxScaler, Binarizer, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.sql.functions import col, length, when
from pyspark.sql.types import DoubleType
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

Сформируем объект конфигурации для Apache Spark, указав необходимые параметры.

In [2]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    user_name = "dchel"
    
    conf = SparkConf()
    conf.setAppName("Lab 2")
    conf.setMaster("local[*]")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    # conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    # conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    # conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [3]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`.

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Загрузка датасета

In [5]:
df = spark.read.parquet("hdfs://namenode:9000/user/dchel/dchel_database/eCommerce_clear_data")

def print_df(df):
    return df.limit(10).toPandas()

In [6]:
print_df(df)

,event_type,product_id,brand,price,contains_appliances,contains_computers,contains_electronics,contains_kitchen,contains_smartphone
0,purchase,3601248,indesit,167.289993,True,False,False,True,False
1,view,8700613,sonar,20.590000,True,False,False,False,False
2,purchase,1005099,samsung,154.410004,False,False,True,False,True
3,cart,1801537,haier,360.079987,False,False,True,False,False
4,view,18300878,thule,110.360001,False,False,False,False,False
5,view,13200717,redford,253.550003,False,False,False,False,False
6,purchase,4804581,xiaomi,37.770000,False,False,True,False,False
7,view,28718040,millionaire,79.019997,False,False,False,False,False
8,view,2701697,bosch,523.020020,True,False,False,True,False
9,purchase,1004785,huawei,276.700012,False,False,True,False,True


Расшифровка столбцов
| Название столбца  | Расшифровка |
| ------------- | ------------- |
| asin | ID товара  |
|reviewerID  |ID рецензента  |
|reviewerName  |Имя рецензента  |
|reviewText  |Текст отзыва  |
|overall  |Общая оценка  (1.0-5.0)|
|summary  |Краткое описание  |
|unixReviewTime  |Время отзыва в Unix-времени  
| reviewTime | Время отзыва ||

Вспомним схему данных

In [7]:
df.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- contains_appliances: boolean (nullable = true)
 |-- contains_computers: boolean (nullable = true)
 |-- contains_electronics: boolean (nullable = true)
 |-- contains_kitchen: boolean (nullable = true)
 |-- contains_smartphone: boolean (nullable = true)



Вычислим количество строк в датафрейме.

In [8]:
df.count()

190323

### Постановка задачи GBT регрессии
Для датасета, заданного представленными колонками, требуется построить модель GBT регрессии 
для оценки рейтинга отзыва `(overall)` по всем остальным признакам.

Для оценки качества обучения следует использовать метрики RMSE и R².

### Подготовка и кодирование признаков

Отделим от датасета некоторую часть объёмом примерно 1000 строк, и сохраним её на диске как локальный csv-файл. Он понадобится в следующей лабораторной работе.

In [9]:
# Отделим от датасета некоторую часть объёмом примерно 1000 строк, и сохраним её на диске как локальный csv-файл
def save_sample_to_csv(data: DataFrame, file_path: str, 
                       sample_size: int = 1000) -> DataFrame:
    """
    Сохраняет первые `sample_size` строк из DataFrame в CSV-файл 
    на драйвере и возвращает DataFrame с оставшимися данными.

    Args:
        data (DataFrame): DataFrame, из которого нужно извлечь
            строки.
        file_path (str): Путь для сохранения CSV-файла.
        sample_size (int): Количество строк для сохранения
            (по умолчанию 1000).

    Returns:
        DataFrame: DataFrame с оставшимися данными.
    """
    # Определяем пропорции для разделения
    sample_fraction = sample_size / data.count()
    remaining_fraction = 1 - sample_fraction

    # Разделяем DataFrame на два непересекающихся набора данных
    sample_data, remaining_data = data.randomSplit(
        [sample_fraction, remaining_fraction]
    )

    # Сохраняем извлеченные строки в CSV-файл на драйвере
    try:
        with open(file_path, mode="w", newline="") as file:
            writer = csv.writer(file)

            # Записываем заголовок
            writer.writerow(data.columns)

            # Записываем строки
            for row in sample_data.take(sample_size):
                writer.writerow(row)
        print(f"Файл \"{file_path}\" с данными успешно создан.")

    except Exception as e:
        print(f"Ошибка при записи файла: {e}")

    return remaining_data

Определяем путь для сохранения csv-файла.

In [10]:
path = "streaming-data.csv"

df = save_sample_to_csv(data=df, file_path=path, sample_size=1000)

Файл "streaming-data.csv" с данными успешно создан.


Оцениваем, сколько строк в датасете осталось

In [11]:
df.count()

189369

Разделим датасет на обучающую и тестовую выборки.

In [12]:
train_df, test_df = df.randomSplit([0.8, 0.2])

In [13]:
print(f"Train dataset size: {train_df.count()}")
print(f"Test dataset size: {test_df.count()}")

Train dataset size: 151450
Test dataset size: 37919


#### Создаем бинарные признаки на основе цены

In [14]:
df = df.withColumn("is_expensive", when(col("price") > 200, 1).otherwise(0))
df = df.withColumn("is_budget", when(col("price") < 50, 1).otherwise(0))
df = df.withColumn("is_mid_range", when((col("price") >= 50) & (col("price") <= 200), 1).otherwise(0))

#### Признаки на основе категорий продуктов

In [15]:
df = df.withColumn("category_count", 
                   col("contains_appliances").cast("int") + 
                   col("contains_computers").cast("int") + 
                   col("contains_electronics").cast("int") + 
                   col("contains_kitchen").cast("int") + 
                   col("contains_smartphone").cast("int"))

#### Признаки на основе типа события

In [16]:
df = df.withColumn("is_purchase", when(col("event_type") == "purchase", 1).otherwise(0))
df = df.withColumn("is_view", when(col("event_type") == "view", 1).otherwise(0))
df = df.withColumn("is_cart", when(col("event_type") == "cart", 1).otherwise(0))

#### Группируем цены в диапазоны

In [17]:
df = df.withColumn("price_range", 
                   when(col("price") < 50, "budget")
                   .when(col("price") < 150, "affordable")
                   .when(col("price") < 300, "premium")
                   .otherwise("luxury"))

# Создаем числовое представление для price_range
df = df.withColumn("price_range_numeric",
                   when(col("price_range") == "budget", 1)
                   .when(col("price_range") == "affordable", 2)
                   .when(col("price_range") == "premium", 3)
                   .otherwise(4))

df.show()

+----------+----------+-------+------------------+-------------------+------------------+--------------------+----------------+-------------------+------------+---------+------------+--------------+-----------+-------+-------+-----------+-------------------+
|event_type|product_id|  brand|             price|contains_appliances|contains_computers|contains_electronics|contains_kitchen|contains_smartphone|is_expensive|is_budget|is_mid_range|category_count|is_purchase|is_view|is_cart|price_range|price_range_numeric|
+----------+----------+-------+------------------+-------------------+------------------+--------------------+----------------+-------------------+------------+---------+------------+--------------+-----------+-------+-------+-----------+-------------------+
|      cart|   1002042|samsung| 77.13999938964844|              false|             false|                true|           false|               true|           0|        0|           1|             2|          0|      0|     

Идентификатор product_id не оказывают влияния на оценку отзыва. Использовать их в модели нет смысла.

Остальные признаки сгруппируем по их типу:

* **Категориальные признаки**: пока отсутствуют в исходных данных
* **Бинарные признаки**: создадим из текстовых данных
* **Количественные признаки**: временные метки и производные от текста
* **Текстовые признаки**: преобразуем в числовые (длина текста)

In [18]:
# Обновим датафреймы train и test с новыми признаками
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
print(train_df.count())
print(test_df.count())

151726
37643


In [19]:
# Определим группы признаков для модели
binary_features = [
    "is_expensive", "is_budget", "is_mid_range", "is_purchase", "is_view", "is_cart"  # Созданные бинарные признаки
]

numeric_features = [
    "price",
    "contains_appliances",
    "contains_computers",
    "contains_electronics",
    "contains_kitchen",
    "contains_smartphone",
    "category_count"
    # "price_range_numeric"
]

categorical_features = ["brand", "event_type", "price_range"]

# Целевая переменная
label_col = "price_range_numeric"

print("Категориальные признаки:", categorical_features)
print("Бинарные признаки:", binary_features)
print("Числовые признаки:", numeric_features)
print("Целевая переменная:", label_col)

Категориальные признаки: ['brand', 'event_type', 'price_range']
Бинарные признаки: ['is_expensive', 'is_budget', 'is_mid_range', 'is_purchase', 'is_view', 'is_cart']
Числовые признаки: ['price', 'contains_appliances', 'contains_computers', 'contains_electronics', 'contains_kitchen', 'contains_smartphone', 'category_count']
Целевая переменная: price_range_numeric


Отсутсвие категориальных признаков никак не помешает LogisticRegressor, поскольку данная модель отличнно работает с только с числовыми признаками и изначально разработана для работы с ними.

Создадим конвейер обработки данных, включающий модель LogisticRegressor

In [20]:
def create_lr_pipeline(categorical_features: list[str], numeric_features: list[str], 
                       binary_features: list[str], label_col: str) -> Pipeline:
    
    stages = []
    
    # Обработка категориальных признаков
    if categorical_features:
        indexed_categorical_features = [f"{feature}_index" for feature in categorical_features]
        onehot_categorical_features = [f"{feature}_ohe" for feature in categorical_features]
        
        string_indexer = StringIndexer(inputCols=categorical_features,
                                      outputCols=indexed_categorical_features,
                                      handleInvalid="keep")
        onehot_encoder = OneHotEncoder(inputCols=indexed_categorical_features,
                                      outputCols=onehot_categorical_features,
                                      dropLast=True,
                                      handleInvalid="keep")
        stages.extend([string_indexer, onehot_encoder])
    
    # Собираем числовые признаки в вектор
    vector_num_assembler = VectorAssembler(inputCols=numeric_features,
                                          outputCol="numeric_vector")
    stages.append(vector_num_assembler)
    
    # Масштабируем числовые признаки (ВАЖНО для LogisticRegression)
    numeric_scaler = StandardScaler(inputCol="numeric_vector",
                                   outputCol="numeric_vector_scaled",
                                   withStd=True,
                                   withMean=True)
    stages.append(numeric_scaler)
    
    # Собираем все features в финальный вектор
    feature_cols = ["numeric_vector_scaled"] + binary_features
    if categorical_features:
        feature_cols.extend(onehot_categorical_features)
    
    vector_all_assembler = VectorAssembler(inputCols=feature_cols,
                                          outputCol="features")
    stages.append(vector_all_assembler)
    
    # Создаем модель LogisticRegression
    lr = LogisticRegression(featuresCol="features",
                           labelCol=label_col,
                           predictionCol="prediction",
                           rawPredictionCol="rawPrediction",
                           probabilityCol="probability",
                           maxIter=100,
                           regParam=0.01,
                           elasticNetParam=0.0,
                           family="multinomial",
                           standardization=False)
    stages.append(lr)
    
    # Создаем конвейер
    pipeline = Pipeline(stages=stages)
    
    return pipeline

In [21]:
pipeline = create_lr_pipeline(categorical_features=categorical_features,
                              numeric_features=numeric_features,
                              binary_features=binary_features,
                              label_col=label_col)
print("Конвейер успешно создан!")

Конвейер успешно создан!


### Обучение модели

Выполним **подбор гиперпараметров** модели LogisticRegressor с помощью кросс-валидации на сетке

Создаем сетку параметров для кросс-валидации, получив объект `LogisticRegressor` из конвейера

In [22]:
# param_grid = ParamGridBuilder() \
#     .addGrid(pipeline.getStages()[-1].maxDepth, [3, 5]) \
#     .addGrid(pipeline.getStages()[-1].maxIter, [50]) \
#     .addGrid(pipeline.getStages()[-1].stepSize, [0.1]) \
#     .build()

lr_model = pipeline.getStages()[-1]

param_grid = ParamGridBuilder() \
    .addGrid(lr_model.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr_model.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(lr_model.maxIter, [10, 100]) \
    .build()

Создаем экземпляр `RegressionEvaluator` для оценки модели

In [23]:
cv_evaluator = MulticlassClassificationEvaluator(
    labelCol=label_col,
    predictionCol="prediction",
    metricName="accuracy"
)

Создаем объект `CrossValidator`

In [24]:
cross_validator = CrossValidator(estimator=pipeline,
                                 estimatorParamMaps=param_grid,
                                 evaluator=cv_evaluator,
                                 numFolds=2,  
                                 parallelism=4)  

Обучаем модель конвейера с использованием кросс-валидации.

In [25]:
print("Начинаем обучение модели LogisticRegressor с кросс-валидацией...")
lr_model = cross_validator.fit(train_df)
print("Обучение завершено!")

Начинаем обучение модели LogisticRegressor с кросс-валидацией...
Обучение завершено!


Выведем параметры **лучшей** модели, определенной в ходе кросс-валидации.

In [26]:
def get_best_lr_params(cv_model: CrossValidatorModel) -> dict[str, float]:
    """
    Получает параметры лучшей модели из объекта CrossValidatorModel для LogisticRegression.

    Args:
        cv_model (CrossValidatorModel): Объект CrossValidatorModel, 
            содержащий лучшую модель.

    Returns:
        Dict[str, float]: Параметры лучшей модели LogisticRegression.
    """
    best_model = cv_model.bestModel
    lr_model = best_model.stages[-1]
    
    best_params = {
        "regParam": lr_model.getRegParam(),
        "elasticNetParam": lr_model.getElasticNetParam(),
        "maxIter": lr_model.getMaxIter(),
        "tol": lr_model.getTol(),
        "fitIntercept": lr_model.getFitIntercept()
    }
    
    # Добавляем family, если доступно
    try:
        best_params["family"] = lr_model.getFamily()
    except:
        best_params["family"] = "multinomial"  # значение по умолчанию
    
    # Добавляем информацию о количестве классов
    try:
        best_params["numClasses"] = lr_model.numClasses
        best_params["numFeatures"] = lr_model.numFeatures
    except:
        pass
        
    return best_params

In [27]:
print("Параметры лучшей модели LogisticRegressor:")
for key, value in get_best_lr_params(cv_model=lr_model).items():
    print(f"{key}: {value}")

Параметры лучшей модели LogisticRegressor:
regParam: 0.01
elasticNetParam: 0.0
maxIter: 100
tol: 1e-06
fitIntercept: True
family: multinomial
numClasses: 5
numFeatures: 1660


### Анализ процесса обучения

In [28]:
# Получаем датасет предсказаний
test_df_predictions = lr_model.transform(test_df)

# Извлекаем список колонок, устанавливаем цену на последнее место
right_columns_order = test_df_predictions.columns
right_columns_order.remove(label_col)
right_columns_order.append(label_col)

# Изменяем последовательность колонок и выводим датафрейм
test_df_predictions = test_df_predictions.select(*right_columns_order)
test_df_predictions.show()

+----------+----------+-------+------------------+-------------------+------------------+--------------------+----------------+-------------------+------------+---------+------------+--------------+-----------+-------+-------+-----------+-----------+----------------+-----------------+------------------+--------------+---------------+--------------------+---------------------+--------------------+--------------------+--------------------+----------+-------------------+
|event_type|product_id|  brand|             price|contains_appliances|contains_computers|contains_electronics|contains_kitchen|contains_smartphone|is_expensive|is_budget|is_mid_range|category_count|is_purchase|is_view|is_cart|price_range|brand_index|event_type_index|price_range_index|         brand_ohe|event_type_ohe|price_range_ohe|      numeric_vector|numeric_vector_scaled|            features|       rawPrediction|         probability|prediction|price_range_numeric|
+----------+----------+-------+------------------+----

In [29]:
def evaluate_model(data: DataFrame, 
                   label_col: str) -> dict[str, float]:
    """
    Оценивает модель с использованием метрик точности, полноты и F1-score.

    Args:
        data: DataFrame, содержащий предсказания и фактические метки.
        label_col: Название колонки с меткой.

    Returns:
        dict[str, float]: Словарь с метриками точности, полноты и F1-score.
    """
    # Вычисляем TP, FP, FN
    tp = data.filter((F.col(label_col) == 1) &
                     (F.col("prediction") == 1)).count()
    fp = data.filter((F.col(label_col) == 0) &
                     (F.col("prediction") == 1)).count()
    fn = data.filter((F.col(label_col) == 1) &
                     (F.col("prediction") == 0)).count()

    # Вычисляем метрики
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0 else 0)

    # Возвращаем словарь с метриками
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [30]:
metrics = evaluate_model(test_df_predictions, label_col)
print(f"Metrics: {metrics}")

Metrics: {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}


### Проверка обобщающей способности модели

Выполним предсказания на тестовой выборке.

Перегруппируем колонки датафрейма, переставив столбец с целевой переменной в конец,  чтобы его значения было удобно сравнивать с предсказанным


In [31]:
print("Выполняем предсказания на тестовой выборке...")
test_df_predictions = lr_model.transform(test_df)

right_columns_order = test_df_predictions.columns
right_columns_order.remove(label_col)
right_columns_order.append(label_col)

# Изменяем последовательность колонок и выводим датафрейм
test_df_predictions = test_df_predictions.select(*right_columns_order)

print("Пример предсказаний (первые 10 строк):")
print_df(test_df_predictions)

Выполняем предсказания на тестовой выборке...
Пример предсказаний (первые 10 строк):


,event_type,product_id,brand,price,contains_appliances,contains_computers,contains_electronics,contains_kitchen,contains_smartphone,is_expensive,...,brand_ohe,event_type_ohe,price_range_ohe,numeric_vector,numeric_vector_scaled,features,rawPrediction,probability,prediction,price_range_numeric
0,cart,1002524,apple,513.469971,False,False,True,False,True,1,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[513.469970703125, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0]","[2.4574649210076784, -0.5690525908165996, -0.3...","(2.4574649210076784, -0.5690525908165996, -0.3...","[-8.40945309560833, -0.2258111535833356, 0.012...","[6.066779809258484e-07, 0.002173045250790688, ...",4.0,4
1,cart,1002536,apple,576.570007,False,False,True,False,True,1,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[576.5700073242188, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0]","[2.900575309228896, -0.5690525908165996, -0.36...","(2.900575309228896, -0.5690525908165996, -0.36...","[-8.409453371461794, -0.39310920659106685, -0....","[4.810249875903076e-07, 0.0014575439962818204,...",4.0,4
2,cart,1002544,apple,460.070007,False,False,True,False,True,1,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[460.07000732421875, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[2.0824718184535045, -0.5690525908165996, -0.3...","(2.0824718184535045, -0.5690525908165996, -0.3...","[-8.409452862160524, -0.0842310470990757, 0.12...","[7.371500978917766e-07, 0.0030419622652994756,...",4.0,4
3,cart,1002629,apple,358.309998,False,False,True,False,True,1,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[358.30999755859375, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[1.367877662139378, -0.5690525908165996, -0.36...","(1.367877662139378, -0.5690525908165996, -0.36...","[-8.409452417297844, 0.18556675999357086, 0.34...","[1.0627282283781796e-06, 0.005743698266945392,...",4.0,4
4,cart,1003114,sony,139.259995,False,False,True,False,True,0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","[139.25999450683594, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[-0.1703675923756715, -0.5690525908165996, -0....","(-0.1703675923756715, -0.5690525908165996, -0....","[-8.409452141646462, 0.362008421179874, 4.8918...","[1.534999268364509e-06, 0.009897034712137482, ...",2.0,2
5,cart,1003304,apple,578.520020,False,False,True,False,True,1,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[578.52001953125, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0]","[2.9142689726935047, -0.5690525908165996, -0.3...","(2.9142689726935047, -0.5690525908165996, -0.3...","[-8.409453379986633, -0.39827930256956234, -0....","[4.775738855883144e-07, 0.001439624623700657, ...",4.0,4
6,cart,1003306,apple,583.169983,False,False,True,False,True,1,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[583.1699829101562, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0]","[2.946922631682869, -0.5690525908165996, -0.36...","(2.946922631682869, -0.5690525908165996, -0.36...","[-8.409453400314806, -0.4106078187084612, -0.1...","[4.6944108979450206e-07, 0.0013977696233556031...",4.0,4
7,cart,1003439,gionee,74.650002,False,False,True,False,True,0,...,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","[74.6500015258789, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0]","[-0.6240814189567402, -0.5690525908165996, -0....","(-0.6240814189567402, -0.5690525908165996, -0....","[-8.409451946912519, 0.5467613269516716, 5.004...","[1.3909770589726976e-06, 0.01078832375207733, ...",2.0,2
8,cart,1004059,xiaomi,378.130005,False,False,True,False,True,1,...,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","[378.1300048828125, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0]","[1.507060

### Сохранение модели

In [32]:
# Путь модели
model_path = f"lr-movies-model"

database_name = "dchel_database"
model_table_name = "gbt_eCommerce_lr_model"

output_path = f"hdfs://namenode:9000/user/dchel/{database_name}/{model_path}"

# Сохраняем модель конвейера в HDFS
try:
    lr_model.bestModel.save(output_path)
    print(f"Модель LogisticRegressor успешно сохранена в \"{output_path}\"")
except Exception as e:
    print(f"Ошибка при сохранении модели: {e}")

Модель LogisticRegressor успешно сохранена в "hdfs://namenode:9000/user/dchel/dchel_database/lr-movies-model"


In [33]:
spark.stop()